# Create Knowledge Graph and Its Features Table for Analysis

## modules

In [4]:
import numpy as np
import torch
import random
import spacy
import pandas as pd
from pykeen.triples.utils import load_triples
from pykeen.triples import TriplesFactory
from pykeen.triples import TriplesNumericLiteralsFactory
from pykeen.datasets import get_dataset
from tqdm import tqdm
from util.databinder import DataBinder

## variable, functions, classes

## Parameters

In [13]:
name_kg = 'fb15k237-with-literals'
dir_kg = './data/raw/fb15k237_with_literals'
dir_save = './data/processed/20240627_fb15k237lit'
name_nlp = 'en_core_web_lg'

## 1. Create triple factory

In [1]:
'''
labeled_triples_train = load_triples(f'{dir_kg}/train.txt')
labeled_triples_test = load_triples(f'{dir_kg}/test.txt')
labeled_triples_valid = load_triples(f'{dir_kg}/valid.txt')
'''

"\nlabeled_triples_train = load_triples(f'{dir_kg}/train.txt')\nlabeled_triples_test = load_triples(f'{dir_kg}/test.txt')\nlabeled_triples_valid = load_triples(f'{dir_kg}/valid.txt')\n"

In [6]:
'''
tf_train = TriplesFactory.from_labeled_triples(labeled_triples_train)
tf_test = TriplesFactory.from_labeled_triples(labeled_triples_test)
tf_valid = TriplesFactory.from_labeled_triples(labeled_triples_valid)
'''
kg = get_dataset(dataset='fb15k237')
tf_train = kg.training
tf_test = kg.testing
tf_valid = kg.validation

INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation


## 2. Create word Embedding

In [9]:
df_text = pd.read_csv(f'{dir_kg}/text_literals.txt', sep='\t', header=None, usecols=[0,2], index_col=0, names=['uri', 'description'])

In [14]:
df_text.loc['/m/03j9ml','description']

"Maggie Roswell is an American film and television actress and voice artist from Los Angeles, California. She is well known for her voice work on the Fox network's animated television series The Simpsons, in which she has played recurring characters such as Maude Flanders, Helen Lovejoy, Miss Hoover, and Luann Van Houten, as well as several minor characters. This work has earned her both an Emmy Award nomination and an Annie Award nomination..Roswell made her acting break-through in the 1980s with appearances in films such as Midnight Madness, Lost in America, and Pretty in Pink, and guest appearances on television shows such as Remington Steele, Masquerade, and Happy Days. She appeared frequently in the sketch comedy The Tim Conway Show from 1980 to 1981, and did voice acting for a few animated films and television shows. Roswell also performed in some theater plays, including one in 1988 directed by Julia Sweeney..In 1989, Roswell was hired for the first season of The Simpsons. She p

In [10]:
dict_tlf = {}
nlp = spacy.load(name_nlp)
for _type, tf in zip(['train', 'test', 'valid'], [tf_train, tf_test, tf_valid]):

    print('\n')
    print(20*'-')
    print(f'start {_type} ...')
    print(20*'-')
    print('\n')
    
    n_entity = tf.num_entities
    dim_embedding = nlp.vocab.vectors_length
    numeric_literals = np.zeros((n_entity, dim_embedding))
    
    list_entity_with_no_lit = []
    list_entity_with_dup_lit = []
    for _id, _label in tqdm(tf.entity_id_to_label.items()):
    
        if _label not in df_text.index:
            list_entity_with_no_lit.append(_label)
            print(f'There is no text corresponding to {_label}({_id}).') 
            continue
        
        if len(df_text.loc[_label]) > 1:
            print(f'{_label} has multiple description. use first one.')
            list_entity_with_dup_lit.append(_label)
            text = df_text.loc[_label, 'description'][0]
        else:
            text = df_text.loc[_label, 'description']
            
        numeric_literals[_id, :] = nlp(text).vector


    tlf = TriplesNumericLiteralsFactory(mapped_triples = tf.mapped_triples, 
                                        entity_to_id = tf.entity_to_id, 
                                        relation_to_id = tf.relation_to_id,
                                        numeric_literals=numeric_literals,
                                        literals_to_id=tf.entity_to_id
                                       )

    dict_tlf[_type] = tlf



--------------------
start train ...
--------------------




  1%|▌                                                                               | 92/14505 [00:02<06:07, 39.25it/s]/tmp/42624858.1.gpu/ipykernel_770626/563545070.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = df_text.loc[_label, 'description'][0]
  1%|▌                                                                              | 100/14505 [00:02<06:36, 36.33it/s]

/m/0123r4 has multiple description. use first one.
/m/01243b has multiple description. use first one.


  1%|█                                                                              | 202/14505 [00:05<05:15, 45.36it/s]

/m/0132_h has multiple description. use first one.


...

 99%|████████████████████████████████████████████████████████████████████████████▎| 14383/14505 [04:40<00:02, 49.53it/s]

/m/0xnc3 has multiple description. use first one.


 99%|████████████████████████████████████████████████████████████████████████████▍| 14395/14505 [04:40<00:02, 50.98it/s]

/m/0xwj has multiple description. use first one.


100%|█████████████████████████████████████████████████████████████████████████████| 14505/14505 [04:43<00:00, 51.20it/s]


In [11]:
dict_tlf

{'train': TriplesNumericLiteralsFactory(num_entities=14505, num_relations=237, create_inverse_triples=False, num_triples=272115, num_literals=14505),
 'test': TriplesNumericLiteralsFactory(num_entities=14505, num_relations=237, create_inverse_triples=False, num_triples=20438, num_literals=14505),
 'valid': TriplesNumericLiteralsFactory(num_entities=14505, num_relations=237, create_inverse_triples=False, num_triples=17526, num_literals=14505)}

## 4. Save data

In [14]:
db = DataBinder(target_dir=dir_save)
db.add('tf_train', tf_train)
db.add('tf_test', tf_test)
db.add('tf_valid', tf_valid)
db.add('tlf_train', dict_tlf['train'])
db.add('tlf_test', dict_tlf['test'])
db.add('tlf_valid', dict_tlf['valid'])
db.add('embedding model', name_nlp)
db.add('knowledge_graph_name', name_kg)

INFO:root:Create ./data/processed/20240627_fb15k237lit
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55
INFO:root:Saved info at 2024-06-27 05:23:55


'fb15k237-with-literals'